## Asyncioライブラリによる非同期処理
+ async : コルーチンを作成
+ await : コルーチン内部で実行. awaitの前後でコルーチンを分割してcpuリソースを一時解放する.
+ ルーチンの種類: 1. メインルーチン(main関数), 2.サブルーチン(関数), 3. コルーチン(中断・再開可能な関数のような実行単位)

In [42]:
import os
import sys
import pathlib
import shutil
import re
import random
import time
import datetime

import asyncio

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline
import japanize_matplotlib

コルーチン

In [43]:
async def coroutine():
    print("僕はコルーチン")
    return

In [44]:
coroutine()

<coroutine object coroutine at 0x0000022F8C4E07C0>

In [45]:
# asyncio.run(coroutine())
await coroutine()

僕はコルーチン


awaitによるコルーチンの分割

In [46]:
async def say_after(delay, what):
    print(f"before_sleep: {what}")
    await asyncio.sleep(delay) # --- [1]
    print(what)

### Awaitableオブジェクト
1. Coroutine
2. Task
3. Future (Taskの低レベルAPI)

In [47]:
async def awaitable_split():
    print(f"started at {time.strftime('%X')}")
    await say_after(1, 'hello')
    await say_after(2, 'world')
    print(f"ended at {time.strftime('%X')}")

In [48]:
# asyncio.run(awaitable_split())
await awaitable_split()

started at 18:26:47
before_sleep: hello
hello
before_sleep: world
world
ended at 18:26:50


+ 上記は、並行処理されていない。理由は、sleep中にReadyなコルーチンがなかったため.
+ 基本的にコルーチンはawaitされて初めてReady状態になる.
+ asyncioで並行処理を実行するためには, awaitをせずにコルーチンをReadyな状態にする必要性がある.
+ TaskでコルーチンをReady状態に状態遷移できる.

### TaskとTaskによる並行処理
+ Taskとは, コルーチンをReady状態にして、実行状態と結果を管理するコントローラの役割
+ Taskは, asyncio.create_task(coroutine())で作成できる. あくまで作成だけ.
+ どこかのコルーチンが一時停止状態になって計算リソースが解放された隙に、初めて処理が実行される.

In [49]:
async def awaitable_split_with_task():
    print(f"started at {time.strftime('%X')}")
    task = asyncio.create_task(say_after(2, 'world'))
    await say_after(1, 'hello')
    await task
    print(f"ended at {time.strftime('%X')}")

In [50]:
# asyncio.run(awaitable_split_with_task()
await awaitable_split_with_task()

started at 18:26:50
before_sleep: hello
before_sleep: world
hello
world
ended at 18:26:52


ただし、コルーチンのReady状態になる順番通りに計算リソースが割り当てられるわけではなく、イベントループの処理ロジックによって割り当てされるため実行順序には注意されたい（経験上、おおよそReady状態になった順番通りにリソースが割り当てられる）。

### asyncioによる並行処理は基本的にTaskを利用することになる
+ 並行化したいものは, とりあえずTaskを作成してawaitする

In [51]:
async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    return f

In [52]:
async def do_asynchronize():
    task_a = asyncio.create_task(factorial('A', 2))
    task_b = asyncio.create_task(factorial('B', 3))
    task_c = asyncio.create_task(factorial('C', 4))

    L = []
    L.append(await task_a)
    L.append(await task_b)
    L.append(await task_c)

    print(L)

In [53]:
# asyncio.run(do_asynchronize())
await do_asynchronize()

Task A: Compute factorial(2), currently i=2...
Task B: Compute factorial(3), currently i=2...
Task C: Compute factorial(4), currently i=2...
Task A: factorial(2) = 2
Task B: Compute factorial(3), currently i=3...
Task C: Compute factorial(4), currently i=3...
Task B: factorial(3) = 6
Task C: Compute factorial(4), currently i=4...
Task C: factorial(4) = 24
[2, 6, 24]


+ Taskが増えるとcreate_task(...)のボイラーテンプレートコードが増える課題がある
+ asyncioは高レベルAPIとしてasyncio.gather()を提供している.
+ JSのPromise.all()のような機能

In [56]:
async def do_gather_asynchronize():
    L = await asyncio.gather(
        factorial('A', 2),
        factorial('B', 3),
        factorial('C', 4),
    )
    print(L)

In [57]:
await do_gather_asynchronize()

Task A: Compute factorial(2), currently i=2...
Task B: Compute factorial(3), currently i=2...
Task C: Compute factorial(4), currently i=2...
Task A: factorial(2) = 2
Task B: Compute factorial(3), currently i=3...
Task C: Compute factorial(4), currently i=3...
Task B: factorial(3) = 6
Task C: Compute factorial(4), currently i=4...
Task C: factorial(4) = 24
[2, 6, 24]
